# **ENGG680 - Introduction to Digital Engineering**
## *Lab Assignment 4: ML Applications - Vehicle Motion State Classification*

## Preliminary: Certificate of Work


*We, the undersigned, certify that this is our own work, which has been done expressly for this course, either without the assistance of any other party or where appropriate we have acknowledged the work of others. Further, we have read and understood the section in the university calendar on plagiarism/cheating/other academic misconduct and we are aware of the implications thereof. We request that the total mark for this assignment be distributed as follows among group members:*

|          | First Name | Last Name | Signature (Full Name, Date) | Hours | Contribution % |
|----------|------------|-----------|-----------------------------|-------|----------------|
| Member 1: | Davis | Dickson | Davis Dickson | 4 | 25% |
| Member 2: | Kaleigh | Beer | Kaleigh Beer | 4 | 25% |
| Member 3: | Sichuang | Nie | Sichuang Nie | 4 | 25% |
| Member 4: | Paul | Dobre | Paul Dobre | 4 | 25% |


---

## Section 1 - Motion Classification Using *All Features* Derived From Inertial Sensor Data (33 Marks)

---

### 1.1 Assignment datasets import (6 Marks)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/engg680_2024_fall/Lab_Assignment4

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/engg680_2024_fall/Lab_Assignment4


---

In [ ]:
# some library imports
import pandas as pd

First load the following datasets into dataframes.
* *UrbanNav-HK_Whampoa-20210521_sensors.csv* to dataframe variable named *cv_set*. This dataset is to be used for model training and hyper parameter tuning.
* *UrbanNav-HK_TST-20210517_sensors.csv* to dataframe variable named *test_set1*. This is first independent test dataset.
* *2020-03-14-16-45-35.csv* to dataframe variable named *test_set2*. This is the second independent test dataset.

In [ ]:
# (2 marks)
# TODO: Load cross validation/training dataset (UrbanNav-HK_Whampoa-20210521_sensors.csv) to 'cv_set' dataframe

cv_set = pd.read_csv('UrbanNav-HK_Whampoa-20210521_sensors.csv')
test_set1 = pd.read_csv('UrbanNav-HK_TST-20210517_sensors.csv')
test_set2 = pd.read_csv('2020-03-14-16-45-35.csv')

For the *cv_set*, look at the dataframe head and info, and print the list of all columns. The *test_set1* and *test_set2* have the same schema as *cv_set*, however, have different number of rows (samples).

In [ ]:
# TODO: Display first two rows of cv_set
# (1 mark)
print(cv_set.head(2))

     ax_sum    ay_sum    az_sum    gx_sum    gy_sum    gz_sum     a_sum  \
0 -0.003929 -0.093081  9.789349 -0.002649  0.001406 -0.001391  9.797418   
1 -0.000966 -0.092417  9.789095 -0.001806  0.000103 -0.001943  9.798815   

      g_sum        ax_var        ay_var  ...     g_min    ax_max    ay_max  \
0  0.022554  8.046094e-07  1.319665e-07  ...  0.000003  0.001496  0.000544   
1  0.021731  1.032619e-06  1.073067e-07  ...  0.000006  0.001865  0.000615   

     az_max    gx_max    gy_max    gz_max     a_max     g_max  motion_state  
0  0.024992  0.000110  0.000075  0.000078  0.024997  0.000187          rest  
1  0.024828  0.000124  0.000098  0.000090  0.024829  0.000144          rest  

[2 rows x 33 columns]


In [ ]:
# TODO: Load the info method of cv_set
# (1 mark)
print(cv_set.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1534 entries, 0 to 1533
Data columns (total 33 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ax_sum        1534 non-null   float64
 1   ay_sum        1534 non-null   float64
 2   az_sum        1534 non-null   float64
 3   gx_sum        1534 non-null   float64
 4   gy_sum        1534 non-null   float64
 5   gz_sum        1534 non-null   float64
 6   a_sum         1534 non-null   float64
 7   g_sum         1534 non-null   float64
 8   ax_var        1534 non-null   float64
 9   ay_var        1534 non-null   float64
 10  az_var        1534 non-null   float64
 11  gx_var        1534 non-null   float64
 12  gy_var        1534 non-null   float64
 13  gz_var        1534 non-null   float64
 14  a_var         1534 non-null   float64
 15  g_var         1534 non-null   float64
 16  ax_min        1534 non-null   float64
 17  ay_min        1534 non-null   float64
 18  az_min        1534 non-null 

In [ ]:
# TODO: Display the number of rows and columns  of all three datasets
# (1 mark)
print(f"'cv_set' has {cv_set.shape[0]} rows and {cv_set.shape[1]} columns")
print(f"'test_set1' has {test_set1.shape[0]} rows and {test_set1.shape[1]} columns")
print(f"'test_set2' has {test_set2.shape[0]} rows and {test_set2.shape[1]} columns")

'cv_set' has 1534 rows and 33 columns
'test_set1' has 783 rows and 33 columns
'test_set2' has 297 rows and 33 columns


In [ ]:
# TODO: Display all columns in cv_set
# (1 mark)
print(cv_set.columns)

Index(['ax_sum', 'ay_sum', 'az_sum', 'gx_sum', 'gy_sum', 'gz_sum', 'a_sum',
       'g_sum', 'ax_var', 'ay_var', 'az_var', 'gx_var', 'gy_var', 'gz_var',
       'a_var', 'g_var', 'ax_min', 'ay_min', 'az_min', 'gx_min', 'gy_min',
       'gz_min', 'a_min', 'g_min', 'ax_max', 'ay_max', 'az_max', 'gx_max',
       'gy_max', 'gz_max', 'a_max', 'g_max', 'motion_state'],
      dtype='object')


In all three dataframes, target vector is stored in the *motion_state* column. All other columns are features.

___

### 1.2 - Grid Search with Cross Validation (18 Marks)

___

#### 1.2.1 - Extract all features and target from cv_set

___

From the *cv_set*, load the target vector stored in the *motion_state* column into a new variable *y*

In [ ]:
# TODO - Load target vector into y
# (1 mark)
y=cv_set['motion_state']

From *cv_set*, load all features, that is all columns except for the target vector, in a new dataframe variable *X*

In [ ]:
# TODO - Load all features from cv_set to X
# (2 marks)
X = pd.DataFrame(cv_set.drop(columns=['motion_state']))
X.head(2)

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,a_min,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max
0,-0.003929,-0.093081,9.789349,-0.002649,0.001406,-0.001391,9.797418,0.022554,8.046094e-07,1.319665e-07,...,0.023871,0.000003,0.001496,0.000544,0.024992,0.000110,0.000075,0.000078,0.024997,0.000187
1,-0.000966,-0.092417,9.789095,-0.001806,0.000103,-0.001943,9.798815,0.021731,1.032619e-06,1.073067e-07,...,0.024153,0.000006,0.001865,0.000615,0.024828,0.000124,0.000098,0.000090,0.024829,0.000144


___

#### 1.2.2 - Configure and Perform Grid Search with Cross-Validation

___

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, Normalizer
from sklearn.model_selection import GridSearchCV

The dataset *(X, y)*, must be split into training/validation *(X_train, y_train)* and testing *(X_test, y_test)* sets. Use the `train_test_split` function from sklearn to create the split, where  **30%** of data is used for testing. Set the `random_state=42`, and `stratify=y`.  

In [ ]:
# TODO - Perform test/train split
# (1 mark)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

Initialize a `Pipeline`, consisting of a *scaler* and a *classifer*, set the default values to *StandardScaler()*, and *SVC()*, respectively.

In [ ]:
# TODO - Initialize Pipeline into variable pipe
# (2 marks)
pipe=Pipeline([('scaler', StandardScaler()), ('classifier', SVC())])

Define the grid search configuration (*param_grid*) for training and hyper parameter tuning of the *support vector classifier*. Tune, the hyper-parameter *C*, over the interval `C=[0.01, 0.1, 1 ,10,100]`, *gamma*, over the interval `gamma=[0.001, 0.01, 0.1,1]`, and use the *radial basis function* as the kernel. For the *scaler* parameter, test with the `StandardScaler`, `RobustScaler`, and `None` options.

In [ ]:
# TODO - Setup grid set
# (4 marks)
C=[0.01, 0.1, 1 ,10,100]
gamma=[0.001, 0.01, 0.1,1]
param_grid = {'scaler': [StandardScaler(), RobustScaler(), None],
              'classifier__C': C,
              'classifier__gamma': gamma,
              'classifier__kernel': ['rbf']}

In [ ]:
# Perform the grid search to train model and tune hyper parameters
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('classifier', SVC())]),
             param_grid={'classifier__C': [0.01, 0.1, 1, 10, 100],
                         'classifier__gamma': [0.001, 0.01, 0.1, 1],
                         'classifier__kernel': ['rbf'],
                         'scaler': [StandardScaler(), RobustScaler(), None]},
             return_train_score=True, scoring='accuracy')

___

#### 1.2.3 - Visualize Grid Search Results

___

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [ ]:
# TODO: Print the results from the grid search
# (2 marks)
best_train_score = grid.cv_results_['mean_train_score'][grid.best_index_]

print("Best params:\n{}\n".format(grid.best_params_))
print("Best cross-validation train score: {:.2f}".format(best_train_score))
print("Best cross-validation validation score: {:.2f}".format(grid.best_score_))
print("Test-set score: {:.2f}".format(grid.score(X_test, y_test)))

Best params:
{'classifier__C': 100, 'classifier__gamma': 0.01, 'classifier__kernel': 'rbf', 'scaler': RobustScaler()}

Best cross-validation train score: 0.98
Best cross-validation validation score: 0.95
Test-set score: 0.96


Display the confusion matrix and print the classification report for the trained model. Use the *(X_test, y_test)* dataset. **Hint** - You can use your code from assignment 3.

In [ ]:
# TODO - Display confusion matrix and print classification report for (X_test, y_test)
# (2 marks)
from sklearn.metrics import confusion_matrix, classification_report
y_pred = grid.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[153   2   0]
 [ 10 192   5]
 [  0   3  96]]
              precision    recall  f1-score   support

        rest       0.94      0.99      0.96       155
    straight       0.97      0.93      0.95       207
        turn       0.95      0.97      0.96        99

    accuracy                           0.96       461
   macro avg       0.95      0.96      0.96       461
weighted avg       0.96      0.96      0.96       461



___

#### 1.2.4 - Questions

1. Comment on the fit of the model to the dataset? (2 marks)
2. In the grid search, are the intervals used for *C* and *gamma* hyperparameters suitable? Why or why not? (2 marks)

#### 1.2.4 - Answers
1. The train and test cross-validation scores are very close (0.98 vs 0.95), indicating that the model is fitting the dataset well and is highly accurate when applied to both train and test data.
2.  They are suitable, as the model was able to find a set of parameters that led to high model performance.

___

### 1.3 - Model Evaluation on Test Datasets 1 and 2 (9 Marks)

___

#### 1.3.1 - Extract all features and targets from test datasets

Next, the model trained in 1.2 will be tested on independent datasets, that is *test_set1* and *test_set2*. <br>
The test_sets must be seperated into features (Xs) and targets (ys).

In [ ]:
# TODO - Load the target vector from test_set1 into y1, and all features from test_set1 into X1
# (1 marks)
y1 = test_set1['motion_state']
X1 = test_set1.drop(columns=['motion_state'])
X1.head(2)

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,a_min,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max
0,0.264465,0.089904,9.786930,-0.002305,0.000218,-0.001143,9.801661,0.025634,0.000001,7.749919e-08,...,0.024056,0.000003,0.002506,0.000845,0.024884,0.000155,0.000078,0.000069,0.024886,0.000170
1,0.264300,0.088882,9.788127,-0.001865,-0.000523,-0.001999,9.802481,0.025218,0.000001,9.243000e-08,...,0.024084,0.000007,0.002427,0.000949,0.024881,0.000201,0.000095,0.000076,0.024883,0.000201


In [ ]:
# TODO - Load the target vector from test_set2 into y2, and all features from test_set2 into X2
y2 = test_set2['motion_state']
X2 = test_set2.drop(columns=['motion_state'])
X2.head(2)

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,a_min,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max
0,-0.046400,0.305907,9.791451,-0.002184,0.001311,-0.001901,9.800683,0.012437,0.000002,3.020959e-07,...,0.048406,0.000007,0.002575,0.002993,0.049701,0.000094,0.000115,0.000080,0.049764,0.000142
1,-0.050073,0.311947,9.785619,-0.003369,0.001898,-0.002051,9.797741,0.013805,0.000003,6.859937e-07,...,0.047696,0.000013,0.003880,0.004106,0.050477,0.000140,0.000140,0.000083,0.050763,0.000151


____

#### 1.3.2 - Visualize Results

For each test dataset, print the test set accuracy score.

In [ ]:
# TODO - Print test score accuracies for test_set1 and test_set2
# (1 marks)

print("test_set1 score: {:.2f}".format(grid.score(X1, y1)))
print("test_set2 score: {:.2f}".format(grid.score(X2, y2)))

test_set1 score: 0.92
test_set2 score: 0.25


Display the confusion matrix and print the classification for test_set 1 *(X1, y1)*

In [ ]:
# TODO - Display confusion matrix and print classification report for (X1, y1)
# (1 marks)
y_pred = grid.predict(X1)
print(confusion_matrix(y1, y_pred))
print(classification_report(y1, y_pred))

[[200  31   0]
 [  7 387  17]
 [  0   7 134]]
              precision    recall  f1-score   support

        rest       0.97      0.87      0.91       231
    straight       0.91      0.94      0.93       411
        turn       0.89      0.95      0.92       141

    accuracy                           0.92       783
   macro avg       0.92      0.92      0.92       783
weighted avg       0.92      0.92      0.92       783



Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [ ]:
# TODO - Display confusion matrix and print classification report for (X2, y2)
# (1 marks)
y_pred = grid.predict(X2)
print(confusion_matrix(y2, y_pred))
print(classification_report(y2, y_pred))

[[  0   0  16]
 [  0   0 206]
 [  0   0  75]]
              precision    recall  f1-score   support

        rest       0.00      0.00      0.00        16
    straight       0.00      0.00      0.00       206
        turn       0.25      1.00      0.40        75

    accuracy                           0.25       297
   macro avg       0.08      0.33      0.13       297
weighted avg       0.06      0.25      0.10       297



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


___

#### 1.3.3 - Questions

**Based on results so far**
1. Comment on the model's performances on the datasets *(X_test, y_test)*, *(X1, y1)*, *(X2, y2)*? A significant difference in performance should be observed, what is this an example of? (3 marks)
1. Provide and explain potential causes for the observed behaviour in performance? (3 marks)

#### 1.3.3 - Answers
1. The model performed very well on test_set1 (X1,y1) with high accuracy of 0.92, but performed very poorly on the second test set (X2,y2) with an accuracy of 0.25. This is an example of the model generalizing poorly to different datasets.
2. The bias could be much different in the sensor data collected for the training set, compared to the bias of the sensor used to collect the second test set. If we look at some of the values in the training set, we can see that they are much more similar to the values in the first test set, which likely explains the difference in performance.

___

## Section 2 - Classification Using *Selected Features* Derived From Inertial Sensor Data (12 Marks)

In this second section, an *SVC* model will be trained and tuned using grid search, however, only *selected features* will be used.

___

### 2.1 Grid Search with Cross Validation (5 Marks)

___

#### 2.1.1 Extract *selected* features and target from *cv_set*

From the *cv_set* dataframe, extract the selected features, listed below, and load them in a new dataframe *X*.
* *ax_var* - variance of accelerometer readings in x
* *ay_var* - variance of accelerometer readings in y
* *az_sum* - sum of accelerometer z readings
* *a_sum* - sum of the norm of accelerometer x, y, z readings
* *gz_var* - variance of gyroscope reading in z

In [ ]:
# TODO - Load selected features from cv_set to X
# (1 marks)
X=cv_set[['ax_var', 'ay_var', 'az_sum', 'a_sum', 'gz_var']]
X.head(2)

,ax_var,ay_var,az_sum,a_sum,gz_var
0,8.046094e-07,1.319665e-07,9.789349,9.797418,7.446916e-10
1,1.032619e-06,1.073067e-07,9.789095,9.798815,6.983481e-10


In [ ]:
# TODO - Load motion_state column (target vector) into y
y=cv_set['motion_state']

___

#### 2.1.2 - Configure and Perform Grid Search with Cross-Validation

Follow the steps from **1.2.2** and perform model training and tuning using the selected feature set.

In [ ]:
# TODO - Perform model training and tuning using `GridSearchCV`
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
pipe=Pipeline([('scaler', StandardScaler()), ('classifier', SVC())])
C=[0.01, 0.1, 1 ,10,100]
gamma=[0.001, 0.01, 0.1,1]
param_grid = {'scaler': [StandardScaler(), RobustScaler(), None],
              'classifier__C': C,
              'classifier__gamma': gamma,
              'classifier__kernel': ['rbf']}
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('classifier', SVC())]),
             param_grid={'classifier__C': [0.01, 0.1, 1, 10, 100],
                         'classifier__gamma': [0.001, 0.01, 0.1, 1],
                         'classifier__kernel': ['rbf'],
                         'scaler': [StandardScaler(), RobustScaler(), None]},
             return_train_score=True, scoring='accuracy')

___

#### 2.1.3 - Visualize Grid Search Results

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [ ]:
# TODO: Print the results from the grid search
best_train_score = grid.cv_results_['mean_train_score'][grid.best_index_]

#print("Best params:\n{}\n".format(???))
print("Best params:\n{}\n".format(grid.best_params_))
#print("Best cross-validation train score: {:.2f}".format(???))
print("Best cross-validation train score: {:.2f}".format(grid.cv_results_['mean_train_score'][grid.best_index_]))
#print("Best cross-validation validation score: {:.2f}".format(???))
print("Best cross-validation validation score: {:.2f}".format(grid.best_score_))
#print("Test-set score: {:.2f}".format(???)))
print("Test-set score: {:.2f}".format(grid.score(X_test, y_test)))

Best params:
{'classifier__C': 100, 'classifier__gamma': 0.01, 'classifier__kernel': 'rbf', 'scaler': RobustScaler()}

Best cross-validation train score: 0.84
Best cross-validation validation score: 0.82
Test-set score: 0.83


Display the confusion matrix and print the classification report for the trained model using the *test* split.

In [ ]:
# TODO - Display confusion matrix and print classification reports
y_pred = grid.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[154   1   0]
 [ 20 173  14]
 [  1  44  54]]
              precision    recall  f1-score   support

        rest       0.88      0.99      0.93       155
    straight       0.79      0.84      0.81       207
        turn       0.79      0.55      0.65        99

    accuracy                           0.83       461
   macro avg       0.82      0.79      0.80       461
weighted avg       0.82      0.83      0.82       461



___

#### 2.1.4 - Question


1. Consider the differences between the performance on the training/cross-validation dataset for section 1 (1.2.3) and section 2 (2.1.3). Which model is a better fit based on *just these two evaluations*? (4 marks)

#### 2.1.4 Answer
1. The first model is a better fit, scoring higher in precision and recall for most actions (resting, going straight, turning). The first model achieved a best training score of 0.98, best validation score of 0.95, and a test score of 0.96. In comparision the second model achieved a best training score of 0.84, a best validation score of 0.82 and a testing score of 0.83.

___

### 2.2 - Model Evaluation on Test Datasets 1 and 2 (7 Marks)

___

#### 2.2.1 - Extract selected features and targets from test datasets

The model trained in 2.1 is to be tested on the independent datasets, that is test_set1 and test_set2.
Seperate the testsets into the selected features and target vectors *(X1, y1)* and (X2,)

In [ ]:
# TODO - Load the target vector from test_set1 into y1, and the selected features from test_set1 into X1
# (1 marks)
y1=test_set1['motion_state']
X1=test_set1[['ax_var', 'ay_var', 'az_sum', 'a_sum', 'gz_var']]
X1.head(2)

,ax_var,ay_var,az_sum,a_sum,gz_var
0,0.000001,7.749919e-08,9.786930,9.801661,7.727190e-10
1,0.000001,9.243000e-08,9.788127,9.802481,8.871433e-10


In [ ]:
# TODO - Load the target vector from test_set2 into y2, and the selected features from test_set2 into X2
y2=test_set2['motion_state']
X2=test_set2[['ax_var', 'ay_var', 'az_sum', 'a_sum', 'gz_var']]
X2.head(2)

,ax_var,ay_var,az_sum,a_sum,gz_var
0,0.000002,3.020959e-07,9.791451,9.800683,1.405038e-09
1,0.000003,6.859937e-07,9.785619,9.797741,1.164381e-09


___

#### 2.2.2 - Visualize Results

For each test dataset, print the test set accuracy score.

In [ ]:
# TODO - Print test score accuracies for test_set1 and test_set2

print("test_set1 score: {:.2f}".format(grid.score(X1, y1)))
print("test_set2 score: {:.2f}".format(grid.score(X2, y2)))

test_set1 score: 0.83
test_set2 score: 0.84


Display the confusion matrix and print the classification for test_set 1 *(X1, y1)*

In [ ]:
# TODO - Display confusion matrix and print classification report for (X1, y1)
y_pred = grid.predict(X1)
print(confusion_matrix(y1, y_pred))
print(classification_report(y1, y_pred))


[[229   2   0]
 [ 26 336  49]
 [  0  57  84]]
              precision    recall  f1-score   support

        rest       0.90      0.99      0.94       231
    straight       0.85      0.82      0.83       411
        turn       0.63      0.60      0.61       141

    accuracy                           0.83       783
   macro avg       0.79      0.80      0.80       783
weighted avg       0.83      0.83      0.83       783



Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [ ]:
# TODO - Display confusion matrix and print classification report for (X2, y2)
y_pred = grid.predict(X2)
print(confusion_matrix(y2, y_pred))
print(classification_report(y2, y_pred))

[[ 15   1   0]
 [  0 178  28]
 [  0  18  57]]
              precision    recall  f1-score   support

        rest       1.00      0.94      0.97        16
    straight       0.90      0.86      0.88       206
        turn       0.67      0.76      0.71        75

    accuracy                           0.84       297
   macro avg       0.86      0.85      0.85       297
weighted avg       0.85      0.84      0.84       297



___

#### 2.2.3 Questions

1. For section 2, compare between the model's performance on the training/validation (2.1) set to the the performance on the two testing sets (2.2). How does the model fit? (3 marks)
1. Compare the models' performances and fits between section 1 and section 2, and consider all three datasets. (3 marks)

#### 2.2.3 Answers
1. The model fits the two test datasets similarly to how it fits the training data. Accuracy scores for each were all between 0.82-0.84. The model shows similar performance for the training/validation and the two testing sets. This shows that the model does not overfit to the data.

2. The model does not fit the first test set as closely as the model used in section 1, but it has much higher performance when applied to the second test set. The model has greater generalizability when applied to different datasets. The training/validation scores are high 0.9 values for model 1, 0.92 for the first test dataset and 0.25 for the second test dataset. This exemplifies the poor generalization ability of the first model. In comparison, the second model performs similarly in training/validation, and testing on both of the datasets with scores around 0.84.

____

## Section 3 - Classification Using *Group Selected Features* From Inertial Sensor Data (10 Marks)

Once again, an *SVC* model is to be trained and tuned using grid search. However, you must select which features to use from the ones provided in *cv_set*.  

____

### 3.1 Grid Search with Cross Validation (4 Marks)

___

#### 3.1.1 Extract *Group Selected* features and target from *cv_set*

**Question**
1. Please list the features that you have selected and explain *why you have chosen* these features? (4 marks)


**Answer**
1.  Using the variances and sum of acceleration and gyroscope signals to classify motions like turn, rest, and straight is a good approach since these features capture the motion’s dynamic changes. Similar features are chosen as in model 2, with the addition of the variances and sum off acceleration and gyroscope signals. The primary reason for this addition is to provide the model more features to hopefully identify the turning classification with more accuracy in comparison to the other models.

In [ ]:
# TODO - Load group selected features from cv_set to X
X =cv_set[['ax_var', 'ay_var', 'az_var',  'gx_var','gy_var','gz_var','a_sum','g_sum']]
X.head(2)

,ax_var,ay_var,az_var,gx_var,gy_var,gz_var,a_sum,g_sum
0,8.046094e-07,1.319665e-07,3.267073e-08,2.365611e-09,6.964705e-10,7.446916e-10,9.797418,0.022554
1,1.032619e-06,1.073067e-07,1.529967e-08,2.242613e-09,6.533136e-10,6.983481e-10,9.798815,0.021731


In [ ]:
# TODO - Load motion_state column (target vector) into y
y =cv_set['motion_state']

___

#### 3.1.2 - Configure and Perform Grid Search with Cross-Validation

Follow the steps from **1.2.2** and perform model training and tuning using the group selected feature set.

In [ ]:
# TODO - Perform model training and tuning using `GridSearchCV`
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
pipe=Pipeline([('scaler', StandardScaler()), ('classifier', SVC())])

C=[0.01, 0.1, 1 ,10,100]
gamma=[0.001, 0.01, 0.1,1]
param_grid = {'scaler': [StandardScaler(), RobustScaler(), None],
              'classifier__C': C,
              'classifier__gamma': gamma,
              'classifier__kernel': ['rbf']}

grid_self = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid_self.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('classifier', SVC())]),
             param_grid={'classifier__C': [0.01, 0.1, 1, 10, 100],
                         'classifier__gamma': [0.001, 0.01, 0.1, 1],
                         'classifier__kernel': ['rbf'],
                         'scaler': [StandardScaler(), RobustScaler(), None]},
             return_train_score=True, scoring='accuracy')

___

#### 3.1.3 - Visualize Grid Search Results

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [ ]:
# TODO: Print the results from the grid search
best_train_score = grid_self.cv_results_['mean_train_score'][grid_self.best_index_]
print("Best params:\n{}\n".format(grid_self.best_params_))
print("Best cross-validation train score: {:.2f}".format(best_train_score))
print("Best cross-validation validation score: {:.2f}".format(grid_self.best_score_))
print("Test-set score: {:.2f}".format(grid_self.score(X_test, y_test)))

Best params:
{'classifier__C': 100, 'classifier__gamma': 0.01, 'classifier__kernel': 'rbf', 'scaler': StandardScaler()}

Best cross-validation train score: 0.95
Best cross-validation validation score: 0.93
Test-set score: 0.93


Display the confusion matrix and print the classification report for the trained model using the *test* split.

In [ ]:
# TODO - Display confusion matrix and print classification report
from sklearn.metrics import confusion_matrix, classification_report
y_pred = grid_self.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[155   0   0]
 [ 25 181   1]
 [  0   4  95]]
              precision    recall  f1-score   support

        rest       0.86      1.00      0.93       155
    straight       0.98      0.87      0.92       207
        turn       0.99      0.96      0.97        99

    accuracy                           0.93       461
   macro avg       0.94      0.94      0.94       461
weighted avg       0.94      0.93      0.94       461



___

### 3.2 - Model Evaluation on Test Datasets 1 and 2 (6 Marks)

___

#### 3.2.1 - Extract group selected features and targets from test datasets

The model trained in 3.1 is to be tested on the independent datasets, that is test_set1 and test_set2.
Seperate the testsets into group selected features and target vectors *(X1, y1)* and *(X2,y2)*

In [ ]:
# TODO - Load the target vector from test_set1 into y1, and the group selected features from test_set1 into X1
y1=test_set1['motion_state']
X1=test_set1[['ax_var', 'ay_var', 'az_var',  'gx_var','gy_var','gz_var','a_sum','g_sum']]
X1.head(2)

,ax_var,ay_var,az_var,gx_var,gy_var,gz_var,a_sum,g_sum
0,0.000001,7.749919e-08,3.920063e-08,3.655217e-09,6.507877e-10,7.727190e-10,9.801661,0.025634
1,0.000001,9.243000e-08,4.112300e-08,3.349120e-09,6.788807e-10,8.871433e-10,9.802481,0.025218


In [ ]:
# TODO - Load the target vector from test_set2 into y2, and the group selected features from test_set2 into X2
y2=test_set2['motion_state']
X2=test_set2[['ax_var', 'ay_var', 'az_var',  'gx_var','gy_var','gz_var','a_sum','g_sum']]
X2.head(2)

,ax_var,ay_var,az_var,gx_var,gy_var,gz_var,a_sum,g_sum
0,0.000002,3.020959e-07,6.324738e-08,1.718706e-09,1.211633e-09,1.405038e-09,9.800683,0.012437
1,0.000003,6.859937e-07,3.110085e-07,2.179497e-09,1.893673e-09,1.164381e-09,9.797741,0.013805


___

#### 3.2.2 - Visualize Results

For each test dataset, print the test set accuracy score.

In [ ]:
# TODO - Print test score accuracies for test_set1 and test_set2
print("test_set1 score: {:.2f}".format(grid_self.score(X1, y1)))
print("test_set2 score: {:.2f}".format(grid_self.score(X2, y2)))

test_set1 score: 0.89
test_set2 score: 0.75


Display the confusion matrix and print the classification for test_set 1 *(X1, y1)*

In [ ]:
# TODO - Display confusion matrix and print classification report for (X1, y1)
y_pred = grid_self.predict(X1)
print(confusion_matrix(y1, y_pred))
print(classification_report(y1, y_pred))

[[184  47   0]
 [ 16 383  12]
 [  0  14 127]]
              precision    recall  f1-score   support

        rest       0.92      0.80      0.85       231
    straight       0.86      0.93      0.90       411
        turn       0.91      0.90      0.91       141

    accuracy                           0.89       783
   macro avg       0.90      0.88      0.89       783
weighted avg       0.89      0.89      0.89       783



Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [ ]:
# TODO - Display confusion matrix and print classification report for (X2, y2)
y_pred = grid_self.predict(X2)
print(confusion_matrix(y2, y_pred))
print(classification_report(y2, y_pred))

[[  5  11   0]
 [  0 163  43]
 [  0  19  56]]
              precision    recall  f1-score   support

        rest       1.00      0.31      0.48        16
    straight       0.84      0.79      0.82       206
        turn       0.57      0.75      0.64        75

    accuracy                           0.75       297
   macro avg       0.80      0.62      0.65       297
weighted avg       0.78      0.75      0.75       297



___

#### 3.2.3 Questions

1. Based on the observations across all three datasets, compare the performance of the model trained in this section to: (3 marks)
    1. Model trained in Section 1 using *all features*.
    1. Model trained in Section 2 using *selected features*.

2. Please proivde potential explanations for the differences observed in models' performances across the three sections. (3 marks)

#### 3.2.3 Answers
1. The model trained in Section 1 using all features achieved an accuracy of 0.92 on the first test dataset and an accuracy of 0.25 on the second test dataset. The model trained in Section 2 achieved an accuracy of 0.83 on the first test dataset and an accuracy of 0.84 on the second test dataset. In comparison the model trained in Section 3 achieved an accuracy of 0.89 on the first test dataset and an accuracy of 0.75 on the second test dataset. This indicates that the model is more effective on the first test dataset than model 2, but less effective than model 1. It is significantly more effective than model 1 on the second test dataset but less effective than model 2. Both model 2 and model 3 seem to struggle with detecting the turning of the car.


2. The model trained on all data shows the impact of too many training features. Using too many features in Section 1 shows how the model overfit to the data and had great performance on the first dataset, but was not able to generalize to the second dataset. Reducing the dimensionality of the features has a clear benefit in the model's ability to generalize to the second test dataset. The third model showcases this to some extent as the addition of the variance features result in improvements on test data 1 and worse performance on test data 2. However, the impact is less drastic than for model 1. This shows the tradeoff in increasing or decreasing features in relation to the model's ability to generalize.

____

## Section 4 - Classification Using Principal Components of Features From Inertial Sensor Data (10 Marks)

____

### 4.1 Principal Component Analysis (PCA) Search with Cross Validation (4 Marks)

___

#### 4.1.1 Extract *ALL* features and target from *cv_set*

In [ ]:
# TODO - Load all features from cv_set to X
X =cv_set.drop(columns=['motion_state'])
X.head(2)

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,a_min,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max
0,-0.003929,-0.093081,9.789349,-0.002649,0.001406,-0.001391,9.797418,0.022554,8.046094e-07,1.319665e-07,...,0.023871,0.000003,0.001496,0.000544,0.024992,0.000110,0.000075,0.000078,0.024997,0.000187
1,-0.000966,-0.092417,9.789095,-0.001806,0.000103,-0.001943,9.798815,0.021731,1.032619e-06,1.073067e-07,...,0.024153,0.000006,0.001865,0.000615,0.024828,0.000124,0.000098,0.000090,0.024829,0.000144


In [ ]:
# TODO - Load motion_state column (target vector) into y
y =cv_set['motion_state']

___

#### 4.1.2 - Configure and Perform Grid Search with Cross-Validation

Similar to the steps from **1.2.2** and perform model training and tuning for an `SVC` model. Perform the `train_test_split`, initiate a `Pipeline`, then define the *Parameters* for `GridSearchCV`. Finally, perfrom the grid search.

In [ ]:
# TODO - Perform model training and tuning using `GridSearchCV`
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In the `Pipeline` add an entry *'pca'* to apply `PCA`.

In [ ]:
# Perform any missing imports
from sklearn.decomposition import PCA # Import PCA
# TODO - Initiate the pipeline
# (1 marks)
pipe =Pipeline([('scaler', StandardScaler()), ('pca',PCA()),('classifier', SVC())])

Similar to previous sections, define the grid search configuration (*param_grid*). Add *n_components* hyper parameter tuning for PCA, and search over the values `n_components=[4,7,11,14,16]`.  

In [ ]:
# TODO - Define the parameter grid for your model
# (3 marks)
C=[0.01, 0.1, 1 ,10,100]
gamma=[0.001, 0.01, 0.1,1]
param_grid = {'scaler': [StandardScaler(), RobustScaler(), None],
              'classifier__C': C,
              'pca__n_components':[4,7,11,14,16],
              'classifier__gamma': gamma,
              'classifier__kernel': ['rbf'],
              }

In [ ]:
# Perform the grid search to train model and tune hyper parameters
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('pca', PCA()), ('classifier', SVC())]),
             param_grid={'classifier__C': [0.01, 0.1, 1, 10, 100],
                         'classifier__gamma': [0.001, 0.01, 0.1, 1],
                         'classifier__kernel': ['rbf'],
                         'pca__n_components': [4, 7, 11, 14, 16],
                         'scaler': [StandardScaler(), RobustScaler(), None]},
             return_train_score=True, scoring='accuracy')

___

#### 4.1.3 - Visualize Grid Search Results

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [ ]:
# TODO: Print the results from the grid search
best_train_score = grid.cv_results_['mean_train_score'][grid.best_index_]
print("Best params:\n{}\n".format(grid.best_params_))
print("Best cross-validation train score: {:.2f}".format(best_train_score))
print("Best cross-validation validation score: {:.2f}".format(grid.best_score_))
print("Test-set score: {:.2f}".format(grid.score(X_test, y_test)))

Best params:
{'classifier__C': 100, 'classifier__gamma': 0.01, 'classifier__kernel': 'rbf', 'pca__n_components': 14, 'scaler': RobustScaler()}

Best cross-validation train score: 0.97
Best cross-validation validation score: 0.94
Test-set score: 0.96


Display the confusion matrix and print the classification report for the trained model using the *test* split.

In [ ]:
# TODO - Display confusion matrix and print classification report
y_pred = grid_self.predict(X2)
print(confusion_matrix(y2, y_pred))
print(classification_report(y2, y_pred))

[[  5  11   0]
 [  0 163  43]
 [  0  19  56]]
              precision    recall  f1-score   support

        rest       1.00      0.31      0.48        16
    straight       0.84      0.79      0.82       206
        turn       0.57      0.75      0.64        75

    accuracy                           0.75       297
   macro avg       0.80      0.62      0.65       297
weighted avg       0.78      0.75      0.75       297



___

### 4.2 - Model Evaluation on Test Datasets 1 and 2 (6 Marks)

___

#### 4.2.1 - Extract group selected features and targets from test datasets

In [ ]:
# TODO - Load the target vector from test_set1 into y1, and all features from test_set1 into X1
y1 =test_set1['motion_state']
X1 =test_set1.drop(columns=['motion_state'])
X1.head(2)

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,a_min,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max
0,0.264465,0.089904,9.786930,-0.002305,0.000218,-0.001143,9.801661,0.025634,0.000001,7.749919e-08,...,0.024056,0.000003,0.002506,0.000845,0.024884,0.000155,0.000078,0.000069,0.024886,0.000170
1,0.264300,0.088882,9.788127,-0.001865,-0.000523,-0.001999,9.802481,0.025218,0.000001,9.243000e-08,...,0.024084,0.000007,0.002427,0.000949,0.024881,0.000201,0.000095,0.000076,0.024883,0.000201


In [ ]:
# TODO - Load the target vector from test_set2 into y2, and all features from test_set2 into X2
y2 = test_set2['motion_state']
X2 =test_set2.drop(columns=['motion_state'])
X2.head(2)

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,a_min,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max
0,-0.046400,0.305907,9.791451,-0.002184,0.001311,-0.001901,9.800683,0.012437,0.000002,3.020959e-07,...,0.048406,0.000007,0.002575,0.002993,0.049701,0.000094,0.000115,0.000080,0.049764,0.000142
1,-0.050073,0.311947,9.785619,-0.003369,0.001898,-0.002051,9.797741,0.013805,0.000003,6.859937e-07,...,0.047696,0.000013,0.003880,0.004106,0.050477,0.000140,0.000140,0.000083,0.050763,0.000151


___

#### 4.2.2 - Visualize Results

In [ ]:
# TODO - Print test score accuracies for test_set1 and test_set2
print("test_set1 score: {:.2f}".format(grid.score(X1, y1)))
print("test_set2 score: {:.2f}".format(grid.score(X2, y2)))

test_set1 score: 0.93
test_set2 score: 0.67


In [ ]:
# TODO - Display confusion matrix and print classification report for (X1, y1)
y_pred = grid.predict(X1)
print(confusion_matrix(y1, y_pred))
print(classification_report(y1, y_pred))

[[223   8   0]
 [ 20 372  19]
 [  0   6 135]]
              precision    recall  f1-score   support

        rest       0.92      0.97      0.94       231
    straight       0.96      0.91      0.93       411
        turn       0.88      0.96      0.92       141

    accuracy                           0.93       783
   macro avg       0.92      0.94      0.93       783
weighted avg       0.93      0.93      0.93       783



Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [ ]:
# TODO - Display confusion matrix and print classification report for (X2, y2)
y_pred = grid.predict(X2)
print(confusion_matrix(y2, y_pred))
print(classification_report(y2, y_pred))

[[  5  11   0]
 [  0 119  87]
 [  0   1  74]]
              precision    recall  f1-score   support

        rest       1.00      0.31      0.48        16
    straight       0.91      0.58      0.71       206
        turn       0.46      0.99      0.63        75

    accuracy                           0.67       297
   macro avg       0.79      0.63      0.60       297
weighted avg       0.80      0.67      0.67       297



___

#### 4.2.3 - Questions

1. Based on the observations across all three datasets, compare the performance of the model with PCA, trained in this section to: (4 marks)
    * Model trained in Section 1 using *all features*.
    * Model trained in Section 2 using *selected features*.
    * Model trained in Section 3 using *group selected features*

2. Please provide potential explanations for the differences observed in models' performances across between section 1 and 4. (2 marks)

#### 4.2.3 - Answers


1. The model trained in Section 1 using all features achieved an accuracy of 0.92 on the first test dataset and an accuracy of 0.25 on the second test dataset. The model trained in Section 2 achieved an accuracy of 0.83 on the first test dataset and an accuracy of 0.84 on the second test dataset. In comparison the model trained in Section 3 achieved an accuracy of 0.89 on the first test dataset and an accuracy of 0.75 on the second test dataset. The model trained in section 4 achieves an accuracy of 0.93 on the first dataset which is the highest and similar to section 1 and an accuracy of 0.67 on the second dataset which is similar to section 3, but far better than section 1. Similarly to the other models, it has difficulty with generalizing the classification of turning with an f1-score of 0.63.

2. Using principal component analysis the number of features is reduced through transforming the full feature set into the principal components which capture a majority of the variance in the data. The model is able to achieve the same performance as in section 1 for the first test dataset, while generalizing far better to the second dataset with an accuracy of 0.67 in comparison to 0.25 for model 1.

____

## Section 5 - Classification Using *Pre Selected Features* Derived From Inertial Sensor Data (10 Marks)

___

Now, a second classifer is to be trained and tuned using grid search. The **selected features from section 2** must be used. You may choose from one of the algorithms discussed that class listed below.
1. Random Forest
1. K-Nearest Neighbors
1. Gradient Boosting Machines
1. Neural Networks

____

### 5.1 Grid Search with Cross Validation (6 Marks)

___

#### 5.1.1 Extract *Selected* features and target from *cv_set*

**Questions**
1. Please list the ML model that you have selected . And explain *why you have chosen* this model? (2 marks)

**Answer**
1.We decided to choose neural network because it can find the inner connections between the observations better. Furthermore, it can better capture non-linear and intricate patterns in data, allowing it to better generalize. We wish to see how it will perform on classifying turning in the second dataset.


In [ ]:
# TODO - Load group selected features from cv_set to X
X=cv_set[['ax_var', 'ay_var', 'az_sum', 'a_sum', 'gz_var']]
X.head(2)

,ax_var,ay_var,az_sum,a_sum,gz_var
0,8.046094e-07,1.319665e-07,9.789349,9.797418,7.446916e-10
1,1.032619e-06,1.073067e-07,9.789095,9.798815,6.983481e-10


In [ ]:
# TODO - Load motion_state column (target vector) into y
y =cv_set['motion_state']

___

#### 5.1.2 - Configure and Perform Grid Search with Cross-Validation

Similar to the steps from **1.2.2** and perform model training and tuning using for the model selected. Perform the `train_test_split`, initiate a `Pipeline`, then define the *Parameters* for `GridSearchCV`. Finally, perfrom the grid search.

In [ ]:
# TODO - Perform train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
# Perform any missing imports
from sklearn.neural_network import MLPClassifier  # Import MLPClassifier
# TODO - Initiate the pipeline
# (1 mark)
pipe =Pipeline([('scaler', StandardScaler()), ('classifier', MLPClassifier())])

In [ ]:
# TODO - Define the parameter grid for your model
# (3 marks)
param_grid = {
    'classifier__hidden_layer_sizes': [(10,), (10, 5), (20, 10)],   # Hidden layers
    'classifier__activation': ['relu', 'tanh'],                    # Activation functions
    'classifier__solver': ['adam', 'sgd'],                         # Optimization algorithms
    'classifier__alpha': [0.0001, 0.001],                          # Regularization (L2 penalty)
    'classifier__learning_rate': ['constant', 'adaptive'],         # Learning rate schedule
    'classifier__max_iter': [200, 300],                            # Maximum iterations
    'scaler': [StandardScaler(), None],                            # With and without scaling
}

In [ ]:
# TODO - Perform the grid search
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid.fit(X_train, y_train)

___

#### 5.1.3 - Visualize Grid Search Results

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [ ]:
# TODO: Print the results from the grid search

best_train_score = grid.cv_results_['mean_train_score'][grid.best_index_]
print("Best params:\n{}\n".format(grid.best_params_))
print("Best cross-validation train score: {:.2f}".format(best_train_score))
print("Best cross-validation validation score: {:.2f}".format(grid.best_score_))
print("Test-set score: {:.2f}".format(grid.score(X_test, y_test)))

Best params:
{'classifier__activation': 'tanh', 'classifier__alpha': 0.001, 'classifier__hidden_layer_sizes': (20, 10), 'classifier__learning_rate': 'constant', 'classifier__max_iter': 300, 'classifier__solver': 'adam', 'scaler': StandardScaler()}

Best cross-validation train score: 0.85
Best cross-validation validation score: 0.84
Test-set score: 0.84


Display the confusion matrix and print the classification report for the trained model using the *test* split.

In [ ]:
# TODO - Display confusion matrix and print classification report
y_pred = grid.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[153   2   0]
 [ 16 171  20]
 [  1  36  62]]
              precision    recall  f1-score   support

        rest       0.90      0.99      0.94       155
    straight       0.82      0.83      0.82       207
        turn       0.76      0.63      0.69        99

    accuracy                           0.84       461
   macro avg       0.82      0.81      0.82       461
weighted avg       0.83      0.84      0.83       461



___

### 5.2 - Model Evaluation on Test Datasets 1 and 2 (4 Marks)

___

#### 5.2.1 - Extract selected features and targets from test datasets

The model trained in 5.1, is to be tested on the independent datasets, that is test_set1 and test_set2.
Seperate the testsets into features and target vectors *(X1, y1)* and *(X2,y2)*

In [ ]:
# TODO - Load the target vector from test_set1 into y1, and the selected features from test_set1 into X1
y1 =test_set1['motion_state']
X1 =test_set1[['ax_var', 'ay_var', 'az_sum', 'a_sum', 'gz_var']]
X1.head(2)

,ax_var,ay_var,az_sum,a_sum,gz_var
0,0.000001,7.749919e-08,9.786930,9.801661,7.727190e-10
1,0.000001,9.243000e-08,9.788127,9.802481,8.871433e-10


In [ ]:
# TODO - Load the target vector from test_set2 into y2, and the selected features from test_set2 into X2
y2 =test_set2['motion_state']
X2 =test_set2[['ax_var', 'ay_var', 'az_sum', 'a_sum', 'gz_var']]
X2.head(2)

,ax_var,ay_var,az_sum,a_sum,gz_var
0,0.000002,3.020959e-07,9.791451,9.800683,1.405038e-09
1,0.000003,6.859937e-07,9.785619,9.797741,1.164381e-09


___

#### 5.2.2 - Visualize Results

For each test dataset, print the test set accuracy score.

In [ ]:
# TODO - Print test score accuracies for test_set1 and test_set2
print("test_set1 score: {:.2f}".format(grid.score(X1, y1)))
print("test_set2 score: {:.2f}".format(grid.score(X2, y2)))

test_set1 score: 0.85
test_set2 score: 0.54


Display the confusion matrix and print the classification for test_set 1 *(X1, y1)*

In [ ]:
# TODO - Display confusion matrix and print classification report for (X1, y1)
y_pred = grid.predict(X1)
print(confusion_matrix(y1, y_pred))
print(classification_report(y1, y_pred))

[[231   0   0]
 [ 18 344  49]
 [  0  51  90]]
              precision    recall  f1-score   support

        rest       0.93      1.00      0.96       231
    straight       0.87      0.84      0.85       411
        turn       0.65      0.64      0.64       141

    accuracy                           0.85       783
   macro avg       0.82      0.83      0.82       783
weighted avg       0.85      0.85      0.85       783



Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [ ]:
# TODO - Display confusion matrix and print classification report for (X2, y2)
y_pred = grid.predict(X2)
print(confusion_matrix(y2, y_pred))
print(classification_report(y2, y_pred))

[[ 15   1   0]
 [ 13  72 121]
 [  1   1  73]]
              precision    recall  f1-score   support

        rest       0.52      0.94      0.67        16
    straight       0.97      0.35      0.51       206
        turn       0.38      0.97      0.54        75

    accuracy                           0.54       297
   macro avg       0.62      0.75      0.57       297
weighted avg       0.80      0.54      0.53       297



___

#### 5.2.3 Question

1. For the model trainied in this section, compare its performance to the performance of SVC which was also trained with the same selected features (section 2). Which model fits better? Explain causes for any observed discrepancies? (4 marks)

#### 5.2.3 - Answer
1.The model trained in Section 2 achieved an accuracy of 0.83 on the first test dataset and an accuracy of 0.84 on the second test dataset. The neural network in section 5 achieved an accuracy of 0.85 on the first test dataset and an accuracy of 0.54 on the second test dataset. The performance of the neural network is disappointing with mediocore results for the accuracy and classifications on both the first test dataset and the second test dataset. From this comparison, it is clear that a neural network is not the optimal model for this problem. This can likely be attributed to the lack of data needed for optimal neural network results. Furthermore, this is a simple enough problem to solve with standard machine learning like the support vector classifier in section 2 and does not need a neural network to model more complex relationships.


____